# Advanced Modeling of Groundwater Flow (GW3099)
![alt text](..\img\header.jpg)

# Exercise 2: MODFLOW 6
The purpose of this exercise is to use MODFLOW 6 to simulate example problem 1 using Flopy.

## Part I. Setup Notebook

In [ ]:
%matplotlib inline
from __future__ import print_function
import sys
import os
import shutil
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import flopy

if '../..' not in sys.path:
    sys.path.append('../..')
import config

print(sys.version)
print('python executable: {}'.format(sys.executable))
print('numpy version: {}'.format(np.__version__))
print('matplotlib version: {}'.format(mpl.__version__))
print('flopy version: {}'.format(flopy.__version__))

## Part II. Create, Run, and Post-Process MODFLOW-2005 Model

In [ ]:
# model info
model_name = 'ex02'
model_ws = './ex02-completed'

# grid properties
nlay = 3
nrow = 21
ncol = 20
delr = 500.
delc = 500.
top = 400.
botm = [220., 200., 0.]

# hydraulic properties
hk0 = 50.
vk0 = 10.
hk1 = 0.01
vk1 = 0.01
hk2 = 200.
vk2 = 20

# boundary conditions
welspd = [[(3 - 1, 11 - 1, 10 - 1), -1.5e5]]
rech = 0.005
rivspd = [[(0, ir, ncol-1), 320., 1.e5, 318.] for ir in range(nrow)]

In [ ]:
# create simulation
sim = flopy.mf6.MFSimulation(sim_name=model_name, version='mf6', exe_name=config.mf6exe, 
                             sim_ws=model_ws)

# create tdis package
tdis_rc = [(1000.0, 1, 1.0)]
tdis = flopy.mf6.ModflowTdis(sim, pname='tdis', time_units='DAYS', 
                             perioddata=tdis_rc)

# create gwf model
gwf = flopy.mf6.ModflowGwf(sim, modelname=model_name,
                           model_nam_file='{}.nam'.format(model_name))
gwf.name_file.save_flows = True

# create iterative model solution and register the gwf model with it
ims = flopy.mf6.ModflowIms(sim, pname='ims', print_option='SUMMARY', 
                           complexity='SIMPLE', outer_hclose=1.e-5, 
                           outer_maximum=100, under_relaxation='NONE', 
                           inner_maximum=100, inner_hclose=1.e-6, 
                           rcloserecord=0.1, linear_acceleration='BICGSTAB', 
                           scaling_method='NONE', reordering_method='NONE', 
                           relaxation_factor=0.99)
sim.register_ims_package(ims, [gwf.name])

# dis
dis = flopy.mf6.ModflowGwfdis(gwf, nlay=nlay, nrow=nrow, ncol=ncol,
                              delr=delr, delc=delc,
                              top=top, botm=botm)

# initial conditions
ic = flopy.mf6.ModflowGwfic(gwf, pname='ic', strt=320.)

# node property flow
npf = flopy.mf6.ModflowGwfnpf(gwf, xt3doptions=[('xt3d')], 
                              icelltype=[1,0,0], 
                              k=[50.0, 0.01, 200.0],
                              k33=[10., 0.01, 20.])

# wel
wel = flopy.mf6.ModflowGwfwel(gwf, print_input=True,
                              stress_period_data=welspd)

# rch
aux = [np.ones((nrow, ncol), dtype=np.int)*6]
rch = flopy.mf6.ModflowGwfrcha(gwf, recharge=0.005, 
                               auxiliary=[('iface',)], 
                               aux={0: [6]})
# riv
riv = flopy.mf6.ModflowGwfriv(gwf, stress_period_data=rivspd)

# output control
oc = flopy.mf6.ModflowGwfoc(gwf, pname='oc', budget_filerecord='{}.cbc'.format(model_name),
                            head_filerecord='{}.hds'.format(model_name),
                            headprintrecord=[('COLUMNS', 10, 'WIDTH', 15,
                                              'DIGITS', 6, 'GENERAL')],
                            saverecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')],
                            printrecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')])

sim.write_simulation()
sim.run_simulation()

In [ ]:
# load the heads and budget files
fname = os.path.join(model_ws, model_name + '.hds')
hdobj = flopy.utils.HeadFile(fname)
head = hdobj.get_data()

fname = os.path.join(model_ws, model_name + '.cbc')
cbb = flopy.utils.CellBudgetFile(fname, precision='double')
#frf = cbb.get_data(text='FLOW RIGHT FACE')[0]
#fff = cbb.get_data(text='FLOW FRONT FACE')[0]

In [ ]:
# plot the results for each layer
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 8))
vmin = head.min()
vmax = head.max()
levels = np.linspace(vmin, vmax, 20)

for ilay in range(nlay):
    ax = axes[ilay]
    #ax = fig.add_subplot(1, nlay, ilay + 1, aspect='equal')
    ax.set_title('Layer {}'.format(ilay + 1))
    ax.set_aspect('equal')
    modelmap = flopy.plot.ModelMap(model=m, layer=ilay, ax=ax)
    quadmesh = modelmap.plot_array(head, masked_values=[999.], alpha=0.5, 
                                   vmin=vmin, vmax=vmax, cmap='jet')
    linecollection = modelmap.plot_grid()
    contour_set = modelmap.contour_array(head, levels=levels, colors='white')
    quiver = modelmap.plot_discharge(frf, fff, head=head)
cb = fig.colorbar(quadmesh, ax=axes, shrink=0.5)

## Part II. Create and Run MODPATH-7 Model

In [ ]:
# particles
partlocs = [(0, i, 0) for i in range(nrow)]
particles = flopy.modpath.Particles.create_particles(partlocs)
pg = flopy.modpath.Particles(particledata=particles)

In [ ]:
mp = flopy.modpath.Modpath7(modelname='ex01_mp', model_ws=model_ws, flowmodel=m, exe_name='mp7')
mpbas = flopy.modpath.Modpath7Bas(mp)
mpsim = flopy.modpath.Modpath7Sim(mp, particlegroups=pg)
mp.write_input()
mp.run_model()

In [ ]:
# load the pathline results
fname = os.path.join(model_ws, 'ex01_mp.mppth')
p = flopy.utils.PathlineFile(fname).get_alldata()

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1, 1, 1)
mm = flopy.plot.ModelMap(model=m)
mm.plot_grid(lw=0.5)
mm.plot_pathline(p, layer='all', colors='k', label='Particle Paths')
#mm.plot_pathline(pw0, layer='all', color='blue', label='captured by wells')
#mm.plot_pathline(pr0, layer='all', color='green', label='captured by rivers')
#mm.plot_endpoint(epd0, direction='starting', colorbar=True)
mm.ax.legend();

## Part III. Create and Run MT3D Model

In [ ]:
# transport boundary conditions
icbund = np.ones((nlay, nrow, ncol), dtype=np.int)
icbund[0, int(nrow / 2), 0] = -1
sconc = np.zeros((nlay, nrow, ncol), dtype=np.float32)
sconc[0, int(nrow / 2), 0] = 100

# observations
rhalf = int(nrow / 2)
chalf = int(ncol / 2)
obs = [(il, rhalf, chalf) for il in range(nlay)]

In [ ]:
modelname = 'ex01_mt3d'
mt = flopy.mt3d.Mt3dms(modelname=modelname, model_ws=model_ws, modflowmodel=m, exe_name='mt3dms')
btn = flopy.mt3d.Mt3dBtn(mt, icbund=icbund, sconc=sconc, perlen=100000., nprs=-1, obs=obs)
adv = flopy.mt3d.Mt3dAdv(mt, mixelm=0)
dsp = flopy.mt3d.Mt3dDsp(mt, al=10.)
ssm = flopy.mt3d.Mt3dSsm(mt)
gcg = flopy.mt3d.Mt3dGcg(mt)
mt.write_input()
mt.run_model(silent=False, normal_msg='Program completed.')

In [ ]:
# load the concentrations
fname = os.path.join(model_ws, 'MT3D001.UCN')
cobj = flopy.utils.UcnFile(fname)
conc = cobj.get_data()

# load observations
fname = os.path.join(model_ws, 'MT3D001.OBS')
cobs = flopy.mt3d.Mt3dms.load_obs(fname)

In [ ]:
# plot the results for each layer
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 8))
a = conc
a = np.ma.masked_less(a, 1.e-4)
vmin = a.min()
vmax = a.max()
levels = [0.01, .1, 1, 10., 100.]

for ilay in range(nlay):
    ax = axes[ilay]
    ax.set_title('Layer {}'.format(ilay + 1))
    ax.set_aspect('equal')
    modelmap = flopy.plot.ModelMap(model=m, layer=ilay, ax=ax)
    quadmesh = modelmap.plot_array(a, masked_values=[999.], alpha=0.5, 
                                   vmin=vmin, vmax=vmax, cmap='jet')
    linecollection = modelmap.plot_grid()
    contour_set = modelmap.contour_array(a, levels=levels, colors='k')
    quiver = modelmap.plot_discharge(frf, fff, head=head)
cb = fig.colorbar(quadmesh, ax=axes, shrink=0.5)

In [ ]:
fig = plt.figure(figsize=(8, 3))
ax = fig.add_subplot(1, 1, 1)
modelxsect = flopy.plot.ModelCrossSection(model=m, line={'row': rhalf})
linecollection = modelxsect.plot_grid()
modelxsect.plot_array(a, head=head, cmap='jet')
t = ax.set_title('Column {}'.format(rhalf + 1))

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
for name in cobs.dtype.names:
    if name.startswith('('):
        ax.plot(cobs['time'], cobs[name], label=name)
ax.legend()
ax.set_xlabel('Time')
ax.set_ylabel('Concentration')
ax.set_title('Simulated Breakthrough Curve');